In [7]:
import sys
path = "Find Strategy"
sys.path.append(path)

from find_mix_strategy import find_mix_strategy

In [10]:
print(find_mix_strategy([[6,3],[10,0]]))

Kỳ vọng của mình: 3.0 
Chiến lược đối thủ: [0. 1. 3.]


In [13]:
print(find_mix_strategy([[2,6],[-1,0]]))

Kỳ vọng của mình: 2.0 
Chiến lược đối thủ: [1. 0. 2.]


In [12]:
print(find_mix_strategy([[0,2],[1,0]]))

Kỳ vọng của mình: 0.6666666666666667 
Chiến lược đối thủ: [0.66666667 0.33333333 0.66666667]


In [11]:
print(find_mix_strategy([[2,6],[-1,0]]))

Kỳ vọng của mình: 2.0 
Chiến lược đối thủ: [1. 0. 2.]


In [8]:
print(find_mix_strategy([[3,2],[2,3]]))

Kỳ vọng của mình: 2.5 
Chiến lược đối thủ: [0.5 0.5 2.5]


In [9]:
print(find_mix_strategy([[2,1],[1,2]]))

Kỳ vọng của mình: 1.5 
Chiến lược đối thủ: [0.5 0.5 1.5]


In [15]:
import numpy as np
from scipy.optimize import linprog

def find_mix_strategy(payoff):
    payoff_extended = [i + [-1] for i in payoff]
    A_ub = np.array(payoff_extended)
    b_ub = np.zeros(len(payoff))
    A_eq = np.array([[1] * len(payoff[0]) + [0]])
    b_eq = np.array([1])
    c = np.array([0] * len(payoff[0]) + [1])
    res = linprog(c, A_ub=A_ub, b_ub=b_ub,bounds=(0, None), A_eq=A_eq,b_eq=b_eq)
    return f"Kỳ vọng của mình: {res.fun} \nChiến lược đối thủ: {res.x}" 


In [52]:
A_ub = np.array([[2,0,0,0,-1,0],[0,1,0,0,-1,0],[0,0,1,0,0,-1],[0,0,0,2,0,-1]])
b_ub = np.zeros(4)
A_eq = np.array([[1,1,0,0,0,0],[0,0,1,1,0,0]])
b_eq = np.ones(2)
c = np.array([0,0,0,0,1,1])
res = linprog(c, A_ub=A_ub, b_ub=b_ub,bounds=(0, None), A_eq=A_eq,b_eq=b_eq)
print("Chiến lược của Nam:",res.x[:2])
print("Chiến lược của Nữ:",res.x[2:4])
print("Kỳ vọng của Nữ:",res.x[4])
print("Kỳ vọng của Nam:",res.x[5])


Chiến lược của Nam: [0.33333333 0.66666667]
Chiến lược của Nữ: [0.66666667 0.33333333]
Kỳ vọng của Nữ: 0.6666666666666666
Kỳ vọng của Nam: 0.6666666666666666


In [77]:
def find_mix_strategy(payoff1, payoff2):
    payoff11 = [i + [0]*len(payoff2) +[-1,0] for i in payoff1]
    payoff21 = [[0]*len(payoff1) +i + [0,-1] for i in payoff2]
    t=payoff11+payoff21
    A_ub = np.array(t)
    b_ub = np.zeros(len(t))
    
    e = [[1]*len(payoff1) + [0]*len(payoff2) +[0,0]] + [[0]*len(payoff1) +[1]*len(payoff2) + [0,0]]
    A_eq = np.array(e)
    b_eq = np.ones(2)
    c = np.array([0] * len(t) + [1,1])
    res = linprog(c, A_ub=A_ub, b_ub=b_ub,bounds=(0, None), A_eq=A_eq,b_eq=b_eq)
    return f"Chiến lược của người 1:{res.x[len(payoff1):len(payoff1)+len(payoff2)]}\n Chiến lược của người 2:{res.x[:len(payoff1)]},\nKỳ vọng của người 1: {res.x[len(payoff1)+len(payoff2):len(payoff1)+len(payoff2)+1]},\nKỳ vọng của người 2: {res.x[len(payoff1)+len(payoff2)+1:len(payoff1)+len(payoff2)+2]}"

In [78]:
payoff1 = [[0,1,1,2,1,1],[1,0,2,1,1,1],[1,1,0,1,1,2],[1,1,1,0,2,1],[2,1,1,1,0,1],[1,2,1,1,1,0]]
payoff2 = [[3,2,2,1,2,2],[2,3,1,2,2,2],[2,2,3,2,2,1],[2,2,2,3,1,2],[1,2,2,2,3,2],[2,1,2,2,2,3]]
print(find_mix_strategy(payoff1,payoff2))

Chiến lược của người 1:[0.33333333 0.         0.         0.33333333 0.33333333 0.        ]
 Chiến lược của người 2:[ 0.33333333 -0.         -0.          0.33333333  0.33333333  0.        ],
Kỳ vọng của người 1: [1.],
Kỳ vọng của người 2: [2.]


In [47]:
def combine_matrices(matrix1, matrix2):
    combined_matrix = [[0, 0, 0, 0] for i in range(4)]
    combined_matrix[0][0] = matrix1[0][0]
    combined_matrix[1][1] = matrix1[1][1]
    combined_matrix[2][2] = matrix2[0][0]
    combined_matrix[3][3] = matrix2[1][1]
    return combined_matrix



result = combine_matrices(matrix1, matrix2)
print(result)


[[2, 0, 0, 0], [0, 1, 0, 0], [0, 0, 1, 0], [0, 0, 0, 2]]


In [44]:

find_mix_strategy([[2,0],[0,1]])

'Kỳ vọng của mình: 0.6666666666666666 \nChiến lược đối thủ: [0.33333333 0.66666667 0.66666667]'